# Introduction

Thank you for your interest in chen and the chennai console. These interactive notebooks introduce chen using the wrapper Python library called `chenpy`. The default docker-compose file from the chen repo starts the chennai console in server mode.

Let's begin by testing the server with a simple `help` command.

In [1]:
from chenpy import client, workspace, utils
from chenpy.detectors import common as atom
from chenpy.logger import console

connection = await client.get("http://chennai:8080", None, "chenadmin", "chenpassword")
res = await client.q(connection, "help")
utils.print_table(res)

val res0: Helper = Welcome to the interactive help system. Below you find a table of all available
top-level commands. To get more detailed help on a specific command, just type

`help.<command>`.

Try `help.importCode` to begin with.
_______________________________________________________________________________________________________________________________________________________________
 command          | description                                                               | example                                                       |
==============================================================================================================================================================|
 annotations      | List annotations                                                          | annotations                                                   |
 callTree         | Show call tree for the given method                                       | callTree(method full name)                                    |
 close            | Close project by name                                                     | close(projectName)                                            |
 cpg              | CPG of the active project                                                 | cpg.method.l                                                  |
 declarations     | List declarations                                                         | declarations                                                  |
 delete           | Close and remove project from disk                                        | delete(projectName)                                           |
 distance         | Show graph edit distance from the source method to the comparison methods | distance(source method iterator, comparison method iterators) |
 exit             | Exit the REPL                                                             |                                                               |
 files            | List files                                                                | files                                                         |
 importAtom       | Create new project from existing atom                                     | importAtom("app.atom")                                        |
 importCode       | Create new project from code                                              | importCode("git url or path")                                 |
 importCpg        | Create new project from existing CPG                                      | importCpg("app.atom")                                         |
 imports          | List imports                                                              | imports                                                       |
 methods          | List methods                                                              | methods('Methods', includeCalls=true, tree=true)              |
 open             | Open project by name                                                      | open("projectName")                                           |
 openForInputPath | Open project for input path                                               |                                                               |
 project          | Currently active project                                                  | project                                                       |
 run              | Run analyzer on active CPG                                                | run.securityprofile                                           |
 save             | Write all changes to disk                                                 | save                                                          |
 sensitive        | List sensitive literals                                                   | sensitive                                                     |
 showSimilar      | Show methods similar to the given method                         

chenpy can interact with chennai server via websockets and seamlessly present the data in the form of tables and trees.

## Analyze a Java application

The best way to learn about chennai is by analyzing a real application. Let's use a simple [vulnerable spring application](https://github.com/HooliCorp/vuln-spring) for this exercise.

There is a convenient `import_code` function in the workspace module for importing code from git and other sources.

In [2]:
await workspace.import_code(connection, "https://github.com/HooliCorp/vuln-spring", project_name="vuln-spring", language="java")

True

In [3]:
console.print(await atom.summary(connection))

Atom Summary      
┏━━━━━━━━━━━━━━┳━━━━━━━┓
┃ Node Type    ┃ Count ┃
┡━━━━━━━━━━━━━━╇━━━━━━━┩
│ Files        │ 4     │
│ Methods      │ 113   │
│ Annotations  │ 37    │
│ Imports      │ 43    │
│ Literals     │ 140   │
│ Config Files │ 2     │
│ Overlays     │ 4     │
└──────────────┴───────┘
      Atom Summary      
┏━━━━━━━━━━━━━━┳━━━━━━━┓
┃ Node Type    ┃ Count ┃
┡━━━━━━━━━━━━━━╇━━━━━━━┩
│ Files        │ 4     │
│ Methods      │ 113   │
│ Annotations  │ 37    │
│ Imports      │ 43    │
│ Literals     │ 140   │
│ Config Files │ 2     │
│ Overlays     │ 4     │
└──────────────┴───────┘

In [4]:
console.print(await atom.files(connection))

Files                                       
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┓
┃ File Name                                                       ┃ Method Count ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━┩
│ src/main/java/com/example/vulnspring/SessionFilter.java         │ 2            │
│ src/main/java/com/example/vulnspring/VulnSpringApplication.java │ 2            │
│ src/main/java/com/example/vulnspring/WebController.java         │ 25           │
└─────────────────────────────────────────────────────────────────┴──────────────┘

In [5]:
console.print(await atom.methods(connection))

Methods                                                                                                              
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ File Name                                                       ┃ Methods                                                                                                                                                      ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ src/main/java/com/example/vulnspring/SessionFilter.java         │ com.example.vulnspring.SessionFilter.doFilter:void(javax.servlet.ServletRequest,javax.servlet.ServletResponse,javax.servlet.FilterChain)                     │
│                                                                 │ com.example.vulnspring.SessionFilter.<init>:void()                                                                                                           │
├─────────────────────────────────────────────────────────────────┼──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┤
│ src/main/java/com/example/vulnspring/VulnSpringApplication.java │ com.example.vulnspring.VulnSpringApplication.main:void(java.lang.String[])                                                                                   │
│                                                                 │ com.example.vulnspring.VulnSpringApplication.<init>:void()                                                                                                   │
├─────────────────────────────────────────────────────────────────┼──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┤
│ src/main/java/com/example/vulnspring/WebController.java         │ com.example.vulnspring.WebController.home:java.lang.String(org.springframework.ui.Model,javax.servlet.http.HttpSession)                                      │
│                                                                 │ com.example.vulnspring.WebController.login:java.lang.String(org.springframework.ui.Model)                                                                    │
│                                                                 │ com.example.vulnspring.WebController.login:java.lang.String(javax.servlet.http.HttpSession,java.lang.String,java.lang.String,org.springframework.ui.Model)   │
│                                                                 │ com.example.vulnspring.WebController.loginSuccess:boolean(java.lang.String,java.lang.String)                                                                 │
│                                                                 │ com.example.vulnspring.WebController.logout:java.lang.String(javax.servlet.http.HttpSession)                                                                 │
│                                                                 │ com.example.vulnspring.WebController.update:java.lang.String(javax.servlet.http.HttpSession,org.springframework.ui.Model)                                    │
│                                                                 │ com.example.vulnspring.WebController.update:java.lang.String(javax.servlet.http.HttpSession,java.lang.String,org.springframework.ui.Model)                   │
│                                                                 │ com.example.vulnspring.WebController.checkDB:java.lang.String(java.lang.String,org.springframework.ui.Model)                                                 │
│                      